In [1]:
from transformers import AutoProcessor, AutoModelForImageTextToText
import json
import outlines
from pydantic import BaseModel, Field

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
processor = AutoProcessor.from_pretrained("google/medgemma-4b-it")
model = AutoModelForImageTextToText.from_pretrained("google/medgemma-4b-it")

In [ ]:
print(processor.tokenizer)

In [ ]:
model - 

In [ ]:
conversation = [
    {
        'role': 'user',
        'content' : [
            {'type' : 'text', 'text' : 'What is the capital of France?'}
        ]
        
    }
]

In [ ]:
class TestOutput(BaseModel):
    answer: str = Field(description = "The answer to the question")
    reasoning: str = Field(description = "The reasoning for the answer")

In [ ]:
inputs = processor.apply_chat_template(
    conversation,
    add_generation_prompt=True,
    tokenize = True,
    return_dict = True,
    return_tensors = 'pt'
    
)


In [ ]:
print(inputs)
print(processor.decode(inputs['input_ids'][0]))

In [ ]:
outputs = model.generate(**inputs)

response = processor.decode(outputs[0], clean_up_tokenization_spaces=True)

print(response)